- 直接求解$Ax = b $,复杂度：$O(n^{3})
- 求解上三角或下三角矩阵，复杂度：$O(n^{2})
# LU分解
$$Ax = b \Rightarrow LUx = b  ( 令Ux = y ) \Rightarrow Ly = b $$
- 原本的$Ax = b$ 的计算复杂度是$O(n^{3})$
- 经过这一些列变换，计算复杂度是一个$O(n^{3})$[高斯消元法，化上三角] 和2个$O(n^{2})$ [上三角求解]

A: symmetric positive definite(SPD,对称正定),可以用$LL^{T}$

对于三对角矩阵，追赶法

In [1]:
using BenchmarkTools

In [2]:
function back_sub(A, b)
    x = zeros(n)
    for i = n:-1:1
        x[i] = b[i]
        for j = i + 1 : n
            x[i] -= A[i, j] * x[j]
        end
        x[i] = x[i] / A[i, i]
    end
    return x
end

back_sub (generic function with 1 method)

In [3]:
function forward_sub(A,b)
    x = zeros(n)
    for i = 1:n
        x[i] = b[i]
        # println("x [$i] =" , x[i])
        for j = 1 : i-1
            x[i] -= A[i, j] * x[j]
        end
        # println("i = ",i)
        x[i] = x[i] / A[i, i]
        # println("最后的x [$i] =" , x[i])
    end
    return x
end

forward_sub (generic function with 1 method)

In [4]:
function LU_Factorization(A,b)
    n = size(A, 1)
    L = zeros(n, n)
    U = copy(A)

    for k = 1:n
        for i = k + 1 : n
            L[i,k] = U[i,k]/U[k,k]
            for j = k : n
                U[i, j] -= L[i,k] * U[k,j]
            end
        end
    end

    for i = 1:n
        L[i,i] = 1
    end
    # println(L)
    # println(U)
    # return L,U
    y = forward_sub(L,b)
    x = back_sub(U, y)
    return x
end

LU_Factorization (generic function with 1 method)

In [6]:
n = 4;
A = [1.0 1.0 2.0 3.0;0.0 2.0 1.0 2.0;1.0 -1.0 2.0 2.0;2.0 2.0 5.0 9.0];
b = [3.0; 1.0; 3.0; 7.0];
LU_Factorization(A,b)

4-element Vector{Float64}:
 1.0
 0.0
 1.0
 0.0

In [8]:
n = 3

A = zeros(n, n)
for i = 1:n
    for j = 1:n
        A[i, j] = randn(1)[1]
    end
end
b = randn(n);
LU_Factorization(A,b)

3-element Vector{Float64}:
  0.5235569989007299
 -0.4397820146008997
  0.7782347713846534

In [10]:
using PyPlot
#随机生成n阶矩阵
n = 100
A = zeros(n, n)
for i = 1:n
    for j = 1:n
        A[i, j] = randn(1)[1]
    end
end
b = randn(n);
# figure()
# spy(A)
# display(gcf())
L,U = LU_Factorization(A,b)
# y = forward_sub(L,b)
# x = back_sub(U, y)

100-element Vector{Float64}:
  0.37035290277421085
 -0.6568298042222591
 -0.8586670005450586
 -1.2400468499437471
  1.9947915461430563
  0.6161142226137745
  1.6338165334602988
 -1.9936180232231162
 -3.1117674178785606
  1.2392475154852354
  ⋮
  0.5302296336515686
  0.7114047587249877
  4.286547327796872
 -3.356690400153187
  1.8116340300731333
  1.1623471362401314
 -0.569534935617437
  1.1041825516298898
 -2.2531916335505557

In [11]:
A\b

100-element Vector{Float64}:
  0.3703529027736924
 -0.6568298042229409
 -0.8586670005450632
 -1.2400468499445407
  1.9947915461441685
  0.6161142226140076
  1.6338165334613401
 -1.9936180232241725
 -3.1117674178801282
  1.2392475154862959
  ⋮
  0.5302296336517407
  0.7114047587251902
  4.286547327798877
 -3.3566904001548123
  1.8116340300744433
  1.162347136240845
 -0.5695349356181254
  1.1041825516305124
 -2.253191633551273

第四次作业
- 写一个lineSolver.jl文件（截图发邮箱）
- drawio.com，

In [12]:
#三对角矩阵求解（追赶法）
function tri_DiagSolver(A,b)
    n = size(A, 1)
    u = zeros(n)
    y = zeros(n)

    u[1] = A[1,2]/A[1,1]
    y[1] = b[1]/A[1,1]
    for i = 2:n-1
        u[i] = A[i,i+1]/(A[i,i]-u[i-1]*A[i,i-1])
        y[i] = ((b[i]-y[i-1]*A[i,i-1])/(A[i,i]-u[i-1]*A[i,i-1]))        
    end
    y[n] = ((b[n]-y[n-1]*A[n,n-1])/(A[n,n]-u[n-1]*A[n,n-1]))
    # 回代求解x
    x = zeros(n)
    x[n] = y[n]
    for i = n-1:-1:1
        x[i] = y[i] - u[i] * x[i+1]
    end
    return x
end

tri_DiagSolver (generic function with 1 method)

In [13]:
A = [1.0 2.0 0.0; 2.0 1.0 1.0; 0.0 1.0 1.0]
b = [1.0, 2.0, 3.0]
tri_DiagSolver(A,b)

3-element Vector{Float64}:
 -0.5
  0.75
  2.25

In [15]:
#随机生成三对角矩阵
using PyPlot

n = 50
A = zeros(n, n)
for i = 1:n
    A[i, i] = randn(1)[1]
    if i>1
        A[i, i-1] = randn(1)[1] 
    end
    if i<n
        A[i,i+1] = randn(1)[1]
    end
end
b = randn(n);
# println(A)
# figure()
# spy(A)
# display(gcf())
tri_DiagSolver(A,b)

50-element Vector{Float64}:
 -1.3721052977118546
  2.520558122074614
 -3.1240730896886153
 -3.990783776155069
 -1.5400024002193753
 30.850292251347195
 21.130675901504123
 24.842107808629077
 -0.8468721957831893
  0.9915850475404206
  ⋮
  6.2429365195131705
  6.4081297383999
 -0.13494169002657375
  0.23767002964714773
  0.04451298404395787
  0.8599611789670886
 -5.813152428581514
  5.806570682789051
 -4.240595770935867

In [16]:
A\b

50-element Vector{Float64}:
 -1.3721052977118506
  2.5205581220746125
 -3.124073089688614
 -3.990783776155067
 -1.5400024002193886
 30.850292251347202
 21.130675901504134
 24.842107808629088
 -0.8468721957831895
  0.9915850475404213
  ⋮
  6.242936519513171
  6.4081297383999
 -0.13494169002657377
  0.23767002964714773
  0.04451298404395817
  0.8599611789670888
 -5.813152428581516
  5.8065706827890535
 -4.240595770935868